## Samba

## Sift

In [ ]:
import pandas as pd
import json
import warnings
import math 
warnings.filterwarnings("ignore")

# load data
data = pd.read_csv('../data/sift.csv').dropna(subset=['Decision Reasons'])

# data = data.head(2) # for testing


def try_float_or_split(s):
    if s is None:
        return
    if isinstance(s, float) and math.isnan(s):
        return None
    
    try:
        return float(s)  # try convert whole string to float
    
    except (ValueError, TypeError):     
        # If not convertible, split and return parts for further processing
        if isinstance(s, str):
            parts = s.strip().split(' ')
            if parts[-1] in ['km/h','km', 'USD', 'hours', 'ms']:
                return float(parts[0])
            if parts[-1] == 'days':
                return float(parts[0])*24
            if parts[-1] == 'minutes':
                return float(parts[0])/60
            else:
                return s
 
        else:
            return None 

## get all possible value for the 'name' field in response -- attributes
def find_all_names(obj, name_values):
    if isinstance(obj, dict):
        for k, v in obj.items():
            if k == 'name':
                name_values.add(v)
            else:
                find_all_names(v, name_values)
    elif isinstance(obj, list):
        for item in obj:
            find_all_names(item, name_values)

attributes = set()

for index, value in data['Decision Reasons'].items():
    if not pd.isna(value):
        try:
            parsed_value = json.loads(value)  # <-- parse JSON string to Python object
            find_all_names(parsed_value, attributes)
        except json.JSONDecodeError:
            print(f"Skipping invalid JSON at index {index}")


## extact value for each attribute
def find_value(obj, target_name):
    if isinstance(obj, dict):
        # Check for keys safely
        if 'name' in obj and obj['name'] == target_name:
            return obj.get('value')
        # Recursively check values inside dict
        for v in obj.values():
            found = find_value(v, target_name)
            if found is not None:
                return found
    elif isinstance(obj, list):
        for item in obj:
            found = find_value(item, target_name)
            if found is not None:
                return try_float_or_split(found)
    return None


for name in attributes:
    data[name] = data['Decision Reasons'].apply(
        lambda x: find_value(json.loads(x), name) if pd.notna(x) else None
    )

## indicators
payment failure, tow/repo, accident

In [22]:
data.isna().mean().sort_values(ascending=False).to_csv('../data/missing.csv')

In [23]:
# ## check if API return chargeback information
# my_string = ''
# for value in data['Decision Reasons']:
#     if not pd.isna(value):
#         my_string = my_string+value
# my_string


In [24]:
# data.to_csv('../data/processed_sift.csv')

In [25]:
# attributes

In [26]:
indicators = pd.read_csv('../data/payment_failure_indicators.csv')


In [27]:

def process_time_interval_data(s):
    if not isinstance(s, str):
        return None  # or np.nan
    parts = s.strip().split(' ')
    if len(parts) != 2:
        return None
    value, unit = parts
    try:
        val = float(value)
    except ValueError:
        return None

    if unit == 'days':
        return val * 24
    elif unit == 'hours':
        return val
    elif unit == 'minutes':
        return val / 60
    else:
        return None


for attri in attributes:
    # print(f"Processing column {attri}")
    num_data = pd.to_numeric(data[attri], errors='coerce')

    if num_data.notna().all():
        # Already fully numeric, assign back
        data[attri] = num_data
        continue

    # Not fully numeric, try special handling:
    unique_vals = data[attri].dropna().unique()
    if len(unique_vals) > 5:
        sample_val = unique_vals[0] if isinstance(unique_vals[0], str) else str(unique_vals[0])
        last_word = sample_val.strip().split(' ')[-1]

        if last_word == 'km':
            data[attri] = data[attri].astype(str).map(lambda x: x.strip().split(' ')[0])
            data[attri] = pd.to_numeric(data[attri], errors='coerce')

        elif last_word in ['days', 'hours', 'minutes']:
            data[attri] = data[attri].map(process_time_interval_data)
            data[attri] = pd.to_numeric(data[attri], errors='coerce')

        else:
            data[attri] = pd.to_numeric(data[attri], errors='ignore')
            print(f"Column {attri} has unknown unit '{last_word}'")

    # else:
    #     print(f"Column {attri} has <=5 unique values, skipping special processing")

Column Successful transactions in the last month (either $sale or $capture) has unknown unit '6.0'
Column Purchase amount in the last day has unknown unit '6090.64'
Column Latest billing address name length has unknown unit '7.0'
Column Browser fingerprint has unknown unit 'ce4df7656e93054dc90ef25b000cab460ff8dd46'
Column Latest payment methods single list item has unknown unit '{"payment_type":"$credit_card","payment_gateway":"$ad:-1315462847'
Column Number of users with the same billing address has unknown unit '6.0'
Column Estimated email address age has unknown unit '350.15999999999997'
Column Mobile events in the last month has unknown unit '20.0'
Column Number of digits in the email username has unknown unit '7.0'
Column Failed transactions rate in the last day has unknown unit '0.583'
Column Latest order amount in USD has unknown unit '1173.0'
Column iOS Device ID has unknown unit 'B6255A9A-399C-4826-84BB-D06FBC0F8C40'
Column Update accounts in the last day has unknown unit '5.0

In [28]:
combined_data = data.merge(indicators, left_on='Account Id', right_on = 'Account Id')

In [29]:
for a in attributes:
    print(a)
    print(combined_data[a].unique())
    print('\n')

Successful transactions in the last month (either $sale or $capture)
[nan  6.  4.  5. 33.  7. 36. 72. 43. 65.]


Purchase amount in the last day
[     nan  6090.64  4309.05  4184.4   4612.09  6378.2   6472.68  6023.78
  4577.35  3119.7   4759.23  4431.84  5700.52  4871.63  6133.1   5909.12
  8167.98  2240.48  3040.87  8006.5   8184.44  2102.76  2408.76  2741.44
  2115.22  4673.7   5993.92  3544.48  4792.32  4168.78  3415.19  2466.96
  2209.56  2414.55  5747.77  7709.51  6189.57  4834.45  5955.4   2763.76
  4176.52  2781.12  5163.24 27666.94  4154.72  6372.8   4698.78  5023.42
  9055.68  5113.36  5203.28 12315.75  6884.32  6051.46  5289.74  3261.44
  2219.28  3342.08  3233.34  5874.87  6011.99  4119.84  5945.7   2207.69
  3623.02  4774.86  3071.16  6075.36  4947.68  5351.38  6841.92  5355.98
  5738.82  7246.6   7267.96  6084.5   7399.89  6034.2 ]


Unique payment fingerprints in the last hour
[nan  2.  4.]


Mobile events in the last minute
[nan  5.  4.]


Latest billing address name le

In [30]:
combined_data[['Abuse Score_y', 'Payment Failure']].groupby('Payment Failure').median()

,Abuse Score_y
Payment Failure,
0,0.04
1,0.05


In [31]:
numeric_cols_df = combined_data.select_dtypes(include='number')
columns = list(set(numeric_cols_df.columns).intersection(set(attributes))) + ['Abuse Score_x']
for num in numeric_cols_df.columns:
    combined_data[num] = combined_data[num].fillna(0)

In [32]:

import pandas as pd
from scipy.stats import ttest_ind, mannwhitneyu
import math

def unpaired_test_per_attribute(df, group_col, attributes, test='t'):
    """
    Perform unpaired statistical test on multiple attributes for two groups,
    and include group means in the result.
    
    Parameters:
    - df: pandas DataFrame containing data
    - group_col: name of the column containing group labels
    - attributes: list of attribute column names to test
    - test: 't' for Welch's t-test, 'mw' for Mann-Whitney U test
    
    Returns:
    - pandas DataFrame with columns:
      attribute, group1_mean, group2_mean, test_statistic, p_value
    """
    groups = df[group_col].dropna().unique()
    if len(groups) != 2:
        raise ValueError(f"Expected exactly 2 groups in '{group_col}', found {len(groups)}: {groups}")
    
    results = []
    for attr in attributes:
        group1_data = df.loc[df[group_col] == groups[0], attr].dropna()
        group2_data = df.loc[df[group_col] == groups[1], attr].dropna()

        # Calculate means, use NaN if no data
        group1_mean = group1_data.mean() if len(group1_data) > 0 else math.nan
        group2_mean = group2_data.mean() if len(group2_data) > 0 else math.nan

        if len(group1_data) == 0 or len(group2_data) == 0:
            stat, p = math.nan, math.nan
        else:
            if test == 't':
                stat, p = ttest_ind(group1_data, group2_data, equal_var=False)
            elif test == 'mw':
                stat, p = mannwhitneyu(group1_data, group2_data, alternative='two-sided')
            else:
                raise ValueError("test must be 't' or 'mw'")
        
        results.append({
            'attribute': attr,
            f'{groups[0]}_mean': group1_mean,
            f'{groups[1]}_mean': group2_mean,
            'test_statistic': stat,
            'p_value': p
        })

    return pd.DataFrame(results)

# def unpaired_test_per_attribute(df, group_col, attributes, test='t'):
#     """
#     Perform unpaired statistical test on multiple attributes for two groups.
    
#     Parameters:
#     - df: pandas DataFrame containing data
#     - group_col: name of the column containing group labels
#     - attributes: list of attribute column names to test
#     - test: 't' for t-test (Welch), 'mw' for Mann-Whitney U test
    
#     Returns:
#     - pandas DataFrame with columns: attribute, test_statistic, p_value
#     """
#     # Extract group labels
#     groups = df[group_col].dropna().unique()
#     if len(groups) != 2:
#         raise ValueError(f"Expected exactly 2 groups in '{group_col}', found {len(groups)}: {groups}")
    
#     results = []
#     for attr in attributes:
#         # Drop NaNs for this attribute & group, to avoid errors in tests
#         group1_data = df.loc[df[group_col] == groups[0], attr].dropna()
#         group2_data = df.loc[df[group_col] == groups[1], attr].dropna()

#         if len(group1_data) == 0 or len(group2_data) == 0:
#             # Not enough data to test
#             stat, p = math.nan, math.nan
#         else:
#             if test == 't':
#                 stat, p = ttest_ind(group1_data, group2_data, equal_var=False)
#             elif test == 'mw':
#                 stat, p = mannwhitneyu(group1_data, group2_data, alternative='two-sided')
#             else:
#                 raise ValueError("test must be 't' or 'mw'")
        
#         results.append({'attribute': attr, 'test_statistic': stat, 'p_value': p})

#     return pd.DataFrame(results)


In [33]:
test_results = unpaired_test_per_attribute(combined_data, 'Payment Failure', columns, test='t')

In [34]:
top_attributes = test_results[test_results['p_value']<0.05]
top_attributes

,attribute,0_mean,1_mean,test_statistic,p_value
1,Purchase amount in the last day,25.355851,82.430156,-4.681177,2.924952e-06
2,Mobile events in the last minute,0.036150,0.010693,5.060756,4.227087e-07
7,Number of users with the same billing address,0.159431,0.104255,4.825870,1.409521e-06
8,Estimated email address age,115.307926,76.601890,3.974512,7.101951e-05
9,Update account names in the last week,0.001390,0.000000,2.183138,2.904354e-02
...,...,...,...,...,...
165,Number of users with similar email address,0.060405,0.323235,-10.528846,1.105425e-25
166,Number of unique payment methods in the past 7...,0.018693,0.004455,5.877310,4.248474e-09
169,Email similarity to billing name,0.002846,0.002037,3.363948,7.716437e-04
170,Number of unique payment methods in the past w...,0.035455,0.069280,-5.392407,7.205272e-08


In [35]:
top_attri_names = top_attributes['attribute'].to_list()
combined_data[top_attri_names+['Payment Failure']].groupby('Payment Failure').mean().T

Payment Failure,0,1
Purchase amount in the last day,25.355851,82.430156
Mobile events in the last minute,0.036150,0.010693
Number of users with the same billing address,0.159431,0.104255
Estimated email address age,115.307926,76.601890
Update account names in the last week,0.001390,0.000000
...,...,...
Number of users with similar email address,0.060405,0.323235
Number of unique payment methods in the past 7 days,0.018693,0.004455
Email similarity to billing name,0.002846,0.002037
Number of unique payment methods in the past week (Update Accounts).,0.035455,0.069280


In [37]:
test_results[test_results['p_value']<0.05].to_csv('../data/sift_ttest_results.csv',index=False)

In [ ]:
#. json data
"Latest payment methods single list item"

In [100]:
import pandas as pd
from scipy.stats import chi2_contingency

def chi_square_per_feature(df, group_col, features):
    results = []
    for feature in features:
        # Contingency table of feature vs group
        contingency = pd.crosstab(df[feature], df[group_col])

        # Skip feature if contingency table invalid (e.g. all one category)
        if contingency.shape[0] < 2 or contingency.shape[1] < 2:
            continue

        chi2, p, dof, expected = chi2_contingency(contingency)

        results.append({
            "feature": feature,
            "chi2_statistic": chi2,
            "p_value": p,
            "degrees_of_freedom": dof
        })

    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values("p_value").reset_index(drop=True)
    return results_df



def fill_nans_in_boolean_columns(df):
    for col in df.columns:
        # select column if dtype is boolean or contains only True/False/NaN
        # We treat columns whose unique non-null values are subset of {True, False}
        unique_vals = set(df[col].dropna().unique())

        if unique_vals.issubset({True, 'false'}):
            # Check if only True or only False present (plus NaNs)
            if unique_vals == {'true'}:  
                # only True + NaN, fill NaN with False or leave? You want to fill NaN with True if only false, opposite if only true:
                # The user said: if only false + NaN, fill NaN with True; if only true + NaN, fill NaN with False.
                # So here unique_vals == {True} means only True + NaN → fill NaN with False
                df[col] = df[col].fillna('false')

            elif unique_vals == {'false'}:  
                # only False + NaN → fill NaN with True
                df[col] = df[col].fillna('true')

            # if both True and False present, do nothing

    return df

In [101]:
cat_cols_df = combined_data.select_dtypes(include='object')
meaningful_cat = []
for col in cat_cols_df.columns:
    if 1 < len(cat_cols_df[col].unique())<= 5: # if the number of unique value in cat data is greater then 1 and no less than 5
        meaningful_cat.append(col)

# filter the meaningful cat data
remove_list = ['Decision', 'Market ID', 'Browser/OS V2', 'Browser/OS V2']
filtered_list = [x for x in meaningful_cat if x not in remove_list]


combined_data = fill_nans_in_boolean_columns(combined_data)

# meaningful_cat 
result_df = chi_square_per_feature(combined_data, "Payment Failure", meaningful_cat)

In [78]:
meaningful_cat

['Decision',
 'Workflow Name',
 'Seller ID',
 'Shipping/credit card country match',
 'Country',
 'Transaction_cvv_result_code',
 'Payment method payment gateway',
 'Latest credit card country',
 'IP/shipping country match',
 'Non-transaction API event without page view or mobile event',
 'Account phone number country code',
 'Latest billing address name casing',
 'Shipping name casing',
 'Latest billing address region',
 'Phone number used when user creates an account operator code ( ocn )',
 'Non-transaction API event without page view',
 'Latest billing address name',
 'IP is Proxy, VPN, or hosting service',
 'Latest billing address address 1',
 'Latest billing address city',
 'Disposable email domain',
 'Browser/OS V2',
 'Latest name casing',
 'IP/billing country codes pair',
 'Latest shipping address name casing',
 'Billing address has lengthy unit number',
 'IP/credit card country match',
 'iOS Mobile Device Name',
 'IP/shipping country codes pair',
 'Android last location neighbo

In [102]:
# result_df[result_df['p_value'] < 0.05]
result_df

,feature,chi2_statistic,p_value,degrees_of_freedom
0,Market ID,32.238966,1.363291e-08,1
1,Decision,34.198528,3.748728e-08,2
2,Browser/OS V2,18.000000,4.398497e-04,3
3,Android Mobile Carrier Country,8.000000,1.831564e-02,2
4,Latest billing address region,8.800000,3.207164e-02,3
5,Billing name casing,4.503810,3.381942e-02,1
6,Country,5.958333,5.083518e-02,2
7,Android Device ID,7.200000,6.578905e-02,3
8,Latest billing address address 1,7.000000,7.189777e-02,3
9,Cookie fingerprint,3.733333,1.546383e-01,2


In [103]:
for col in meaningful_cat:
    print(pd.crosstab(combined_data[col], combined_data['Payment Failure']))
    print('\n')

Payment Failure      0     1
Decision                    
APPROVE          10735  3548
NONE                28    10
REVIEW            2183   931


Payment Failure                0     1
Workflow Name                         
Prod_Payments_Risk         12286  4284
Prod_Payments_Risk_202508    660   205


Payment Failure  0  1
Seller ID            
FLEXCAR          4  4


Payment Failure                         0     1
Shipping/credit card country match             
True                                12918  4481
false                                  28     8


Payment Failure     0  1
Country                 
India              10  0
Italy               1  1
Republic of Korea   1  0


Payment Failure              0  1
Transaction_cvv_result_code      
N                            8  2
P                            0  1


Payment Failure                   0   1
Payment method payment gateway         
$stripe                         177  41


Payment Failure              0   1
Latest cred

In [99]:
set(combined_data['Non-transaction API event without page view'].dropna().unique())

{'true'}